# CMU Movie data

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os

from src.data_completion import *
from src.data_preprocessing import *
from src.data_loading import *
from src.data_fetching import *
from src.data_visualization import *

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

## Download datasets
The following code dowloads the datasets that were too heavy to be included, and places them directly in the `data` folder

In [ ]:
# Download TMDb Dataset
download_tmdb()

# Download IMDb Dataset
download_imdb("name.basics.tsv")
download_imdb("title.ratings.tsv")
download_imdb("title.principals.tsv")

# Download CMU dataset
download_cmu()


## Initial data inspection
We will first try to provide a first generic inspection of the CMU movie dataset we decided to work on.

### Load Data
The dataset is divided in 3 parts, the characters, the movies and the plots of the movies.

In [ ]:
character_data, movie_data, plot_data = load_data()

### Characters dataset
Let's first have a look to the character dataset.

In [ ]:
print(f'There are {character_data.shape[0]} characters with {character_data.shape[1]} features for each.')

In [ ]:
character_data.head(2)

We directly note that the actor ethnicities are not human readable, they look to be freebase ids. It's something we will need to fix during preprocessing.
Let's now see how much missing data we have.

In [ ]:
print("Percentage of null rows in the characters dataset for each features:")
print(character_data.isnull().mean().round(3)*100)

We see that we miss a lot of character names/ids, actor heights, ethnicity and age at release. We can also check for duplicate rows.

In [ ]:
print(f"Duplicated rows: {character_data.duplicated().sum()}")

### Movies dataset
Let's now have a look at the movies dataset.

In [ ]:
print(f'There are {movie_data.shape[0]} movies with {movie_data.shape[1]} features for each.')

In [ ]:
movie_data.head(2)

Let's now see if we have a lot of missing data. We will also verify that we don't have duplicated rows.

In [ ]:
print("Percentage of null rows in the movies dataset for each features:")
print(movie_data.isnull().mean().round(3)*100)

In [ ]:
print(f"Duplicated rows: {movie_data.duplicated().sum()}")

Ouch! We only have box office revenue for 10% of our movies, that's not good news since it's a key feature in our research problematic, we will need to fix this. Apart from this, we can also note that we are missing 25% of the runtime information. We could try to improve this. This also applies to the movie release data. For the languages, countries and genres, we note that they are dictionaries meaning that we first need to preprocess them a bit (for example transforming them to a list) to then be able to see the percentage of missing data. We will do it now:

In [ ]:
# Extract the readable values for 'languages', 'countries', and 'genres' columns. Also clean the language column.

movie_data['languages'] = movie_data['languages'].apply(lambda x: extract_values(x, clean_func=clean_language))
movie_data['countries'] = movie_data['countries'].apply(lambda x: extract_values(x)) 
movie_data['genres'] = movie_data['genres'].apply(lambda x: extract_values(x))  

We can now have a look to the missing data:

In [ ]:
# Calculate the number of None (NaN) values for each column
none_languages = movie_data['languages'].isna().mean()
none_countries = movie_data['countries'].isna().mean()
none_genres = movie_data['genres'].isna().mean()

# Print the counts of None (NaN) values
print(f"Percentage of None values in 'languages': {none_languages:.2%}")
print(f"Percentage of None values in 'countries': {none_countries:.2%}")
print(f"Percentage of None values in 'genres': {none_genres:.2%}")

We miss some data but nothing too huge.

### Plot summary dataset
Let's now have a look at the plot summaries dataset.

In [ ]:
print(f'There are {plot_data.shape[0]} plot summaries with {plot_data.shape[1]} features for each.')

In [ ]:
plot_data.head(2)

Let's see if we have some rows that are invalid (no summary or wikipedia id).

In [ ]:
print("Pourcentage of null rows in the plot summaries dataset:")
print(plot_data.isnull().mean().round(3)*100)

Good new, we have nothing missing here.

## Data completion

### Missing data
Before going deeper to the analysis, we want to already fix the fact that we are missing data for most of the box office results and some runtime values.

Let's then first to get more data on box office results to decrease the number of missing data we have for now. To do this, we will merge the current dataset with a dataset that contains information about 1,000,000 movies collected from The Movie Database (TMDb), including revenue and runtime (https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies?resource=download).

In [ ]:
# Load the new dataset (and rename some columns)
movies_dataset = pd.read_csv('data/tmbd_movies.csv')
movies_dataset['box_office_revenue'] = pd.to_numeric(movies_dataset['revenue'], errors='coerce') 
movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], errors='coerce')

# We remove the nan of movie release date since we merge on it
movie_data['movie_release_date'] = pd.to_datetime(movie_data['movie_release_date'], errors='coerce')
movie_data = movie_data.dropna(subset=['movie_release_date'])

movies_dataset.head(2)

In [ ]:
movie_data_merged, before_missing, after_missing = merge_for_completion(movie_data, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "box_office_revenue", merge_strategy='prioritize_first')

In [ ]:
# Print the before and after missing percentages
print(f"Box office results missing percentage before merge (on title) with wikidata: {before_missing:.2%}")
print(f"Box office results missing percentage after merge (on title) with wikidata: {after_missing:.2%}")

Big improvement, that's good. Let's try to improve the runtime data.

In [ ]:
movie_data_merged, before_missing, after_missing = merge_for_completion(movie_data_merged, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "runtime", merge_strategy='prioritize_first')

In [ ]:
# Print the before and after missing percentages
print(f"Runtime results missing percentage before merge (on title) with The Movies Dataset: {before_missing:.2%}")
print(f"Runtime results missing percentage after merge (on title) with The Movies Dataset: {after_missing:.2%}")

That's pretty cool too, we also want to merge the IMBD ID column because we need it afterwards (and we don't have it with the CMU dataset).

In [ ]:
movie_data_merged, _, _ = merge_for_completion(movie_data_merged, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "imdb_id", merge_strategy='add_column')

### Adding the rating and lead actors of movies

Now we want to merge with the IMDb datasets (https://developer.imdb.com/non-commercial-datasets/) in order to obtain ratings and lead actors. We consider an actor to be a lead actor if their ordering is 1 or 2.

In [ ]:
# Load IMDb ratings and select relevant columns
imdb_ratings = pd.read_csv('data/title.ratings.tsv', sep='\t')
imdb_ratings = imdb_ratings.rename(columns={'tconst': 'imdb_id'})
imdb_ratings = imdb_ratings[['imdb_id', 'averageRating', 'numVotes']]

In [ ]:
# Load IMDb names data for actors
imdb_names = pd.read_csv('data/name.basics.tsv', sep='\t')

In [ ]:
# Extract lead actors (representing the first and second roles)
filtered_lead_actors = extract_lead_actors()

In [ ]:
# Merge the lead actors into movie data
movie_data_merged = merge_lead_actors_and_ratings(movie_data_merged, imdb_names, imdb_ratings, filtered_lead_actors)
movie_data_merged.head(2)

In [ ]:
print(f"We have {movie_data_merged.shape[0]} rows in our merged movies dataset.")

## Cleaning and removing outliers

Before analyzing the data any further, let's clean our data and remove outliers.

### Character dataset

First thing we want to do is to make our ethnicites human readable. To do it we used the wikidata API to map the IDs. We created a CSV file to avoid making multiple request each time we run the notebook.

In [ ]:
merged_character_data = map_ethnicities(character_data)

We can now do more basic preprocessing. For more details, please have a look to our function in src/data_preprocessing.py. 

In [ ]:
character_data_valid = preprocess_characters(merged_character_data)

In [ ]:
# Print the final dataset size
print(f"Number of characters after preprocessing: {character_data_valid.shape[0]}.")

In [ ]:
# Check that no nan values still there
character_data_valid.isnull().mean()

This is good. We don't care about the movie_release_date here.

### Movies dataset

We will also remove the outliers and preprocess the movie dataset.

In [ ]:
movie_data_valid = preprocess_movies(movie_data_merged)

In [ ]:
print(f"Number of movies after preprocessing: {movie_data_valid.shape[0]}.")

In [ ]:
# Check that no nan values still there
movie_data_valid.isnull().mean()

That looks perfect!

## Movies subset creation with complete lead actors data 

Now that we have the two main actors of each movie, let's see if we have corresponding characteristics in the preprocessed character dataset. First we will extract the characters data of all our lead actors in all our films.

In [ ]:
lead_actor_data = extract_movies_with_lead_actors_data(movie_data_valid, character_data_valid)

No missing value makes sense since we preprocessed our character data. Let's try now to see how much data we have on our lead actors.

In [ ]:
print(f"We have data for {lead_actor_data.shape[0]/(movie_data_valid["lead_actor_1"].notna().sum() + movie_data_valid["lead_actor_2"].notna().sum())*100}% ({lead_actor_data.shape[0]} actors) of our lead actors")

Ok good, let's now create a subset dataframe that will contains only the movies for which we have complete data on the lead actors.

In [ ]:
subset_movies_with_lead_actors_data = movie_data_valid[movie_data_valid['freebase_movie_id'].isin(lead_actor_data['freebase_movie_id'])]

In [ ]:
print(f"We have {subset_movies_with_lead_actors_data.shape[0]} movies for which we have full data on our lead actors")

Now that we have this movies subset, we can extract the corresponding lead actors and characters subsets for these movies.

In [ ]:
# Filter the character_data_valid dataset to keep only rows with freebase_movie_id present in movie_data_extracted
subset_characters_with_lead_actor_data = character_data_valid[character_data_valid['freebase_movie_id'].isin(subset_movies_with_lead_actors_data['freebase_movie_id'])]

# Extract the relevant columns for characters and associated movies
subset_characters_with_lead_actor_data = subset_characters_with_lead_actor_data[['actor_name', 'actor_dob', 'actor_gender', 'actor_ethnicity_label', 'actor_height', 'actor_age_at_release', 'freebase_movie_id', 'character_name']]

# Display the first few rows of the filtered and extracted data
subset_characters_with_lead_actor_data.head(2)


In [ ]:
print(f"We have {subset_characters_with_lead_actor_data.shape[0]} characters for the movies subset.")

In [ ]:
subset_lead_actors = extract_movies_with_lead_actors_data(subset_movies_with_lead_actors_data, character_data_valid)

In [ ]:
print(f"We have {subset_lead_actors.shape[0]} lead actors for the movies subset.")

## Saving our newly created dataframes

We will save our five dataframes:

- full_movie_data_preprocessed that contains all the movies preprocessed 
- full_characters_data_preprocessed that contains all the characters preprocessed
- subset_movies_with_lead_actors_data that contains movies where we have data on all the lead actors
- lead_actors_data_on_subset_movie that contains information on the lead actors of the previous subset
- character_data_valid_filtered that contains information on all the characters of the previous subset

In [ ]:
preprocessed_path = 'data/preprocessed'

# Ensure the directory exists
os.makedirs(preprocessed_path, exist_ok=True)

preprocessed_names = {
    'full_movie_data_preprocessed.csv': movie_data_valid,
    'full_characters_data_preprocessed.csv': character_data_valid,
    'subset_movie_with_full_data_on_lead_actors.csv': subset_movies_with_lead_actors_data,
    'lead_actors_data_on_subset_movie.csv': subset_lead_actors,
    'characters_data_on_subset_movie.csv': subset_characters_with_lead_actor_data
}

for name, df in preprocessed_names.items():
    df.to_csv(os.path.join(preprocessed_path, name), index=False)

## Loading our preprocessed dataframes

We can skip all the preprocessing process and load our dataframes directly here:

In [ ]:
# Load the preprocessed datasets
full_movie_data_preprocessed = pd.read_csv('data/preprocessed/full_movie_data_preprocessed.csv')
full_characters_data_preprocessed = pd.read_csv('data/preprocessed/full_characters_data_preprocessed.csv')
subset_movie_with_full_data_on_lead_actors = pd.read_csv('data/preprocessed/subset_movie_with_full_data_on_lead_actors.csv')
lead_actors_data_on_subset_movie = pd.read_csv('data/preprocessed/lead_actors_data_on_subset_movie.csv')
characters_data_on_subset_movie = pd.read_csv('data/preprocessed/characters_data_on_subset_movie.csv')

# We must convert dates to datetime
lead_actors_data_on_subset_movie['actor_dob'] = pd.to_datetime(lead_actors_data_on_subset_movie['actor_dob'])
characters_data_on_subset_movie['actor_dob'] = pd.to_datetime(characters_data_on_subset_movie['actor_dob'])
full_movie_data_preprocessed['movie_release_date'] = pd.to_datetime(full_movie_data_preprocessed['movie_release_date'])


## Deeper analysis
Now that our data is more complete, we can do a more in deep analysis.

### Lead actors dataset 

Let's first analyze our dataframe with the lead actors just created. We will start with a summary of the statistics of the numerical features.

In [ ]:
lead_actors_data_on_subset_movie.describe()

Let's print their distributions:

In [ ]:
histogram_actors(lead_actors_data_on_subset_movie)

Let's now explore the categorical data.

In [ ]:
count_actors(lead_actors_data_on_subset_movie)

We can visualize here the correlation matrix between the numerical attributes of the actors.

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

lead_actors_for_corr = lead_actors_data_on_subset_movie[['actor_height', 'actor_age_at_release', 'actor_dob']]
# Convert gender to boolean by getting dummies
lead_actors_gender_dummies = pd.get_dummies(lead_actors_for_corr['actor_gender'])
lead_actors_for_corr.drop('actor_gender', axis=1, inplace=True)
lead_actors_for_corr = lead_actors_for_corr.join(lead_actors_gender_dummies)
# lead_actors_for_corr['F'] = lead_actors_gender_dummies['F']
# lead_actors_for_corr['M'] = lead_actors_gender_dummies['M']
corr_matrix = lead_actors_for_corr.corr()
label_names = ['Height', 'Age at release']
ax = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', xticklabels=label_names, yticklabels=label_names)
ax.set_title('Heatmap of some actors attributes');


### Movies dataset 

Let's now analyse our movies dataset. We will start with a summary of the statistics of the numerical features.

In [ ]:
movie_data_completed = full_movie_data_preprocessed.copy()
movie_data_completed.describe()

Let's print their distributions (except for the wikipedia id):

In [ ]:
histogram_movies(movie_data_completed)

Let's now look at the ratings and number of votes per movie.

In [ ]:
histogram_movie_ratings(movie_data_completed)

We can now have a look at the categorical features:

In [ ]:
count_movies(movie_data_completed)

As we can expect, the English language and United States of America clearly dominate the movie industry.

### Characters dataset

We now analyze the dataframe with the characters (including non-lead actors). We again start with a summary of the statistics of the numerical features.

In [ ]:
characters_data_on_subset_movie.describe()

Let's do some plots to visualize better. As it is very similar as the plots from the lead actors dataset, we will compare our plots with the ones from this dataset.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

histogram_actors(characters_data_on_subset_movie, axes=axes)
histogram_actors(lead_actors_data_on_subset_movie, axes=axes)

fig.tight_layout()
fig.legend(["All actors", "Lead actors"], loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2)


Finally we compare the gender distribution between all actors and only lead actors.

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
width = 0.4

gender_dist_all = characters_data_on_subset_movie['actor_gender'].value_counts(normalize=True) * 100
gender_dist_lead = lead_actors_data_on_subset_movie['actor_gender'].value_counts(normalize=True) * 100
labels = list(gender_dist_all.keys())

bar_all = ax.bar(labels, gender_dist_all, -width, label="All actors", align='edge')
bar_lead = ax.bar(labels, gender_dist_lead, width, label="Lead actors", align='edge')

ax.set_title("Comparison of actors' gender distribution between lead roles and all roles")
ax.set_xlabel("Gender")
ax.set_ylabel("Percent")
plt.grid(axis='y', linestyle='--', alpha=0.5)

ax.legend()
plt.show()

### Low vs. high rating/revenue analysis

Now let's see at the difference between low and high rated movies as well as between movies with low and high box office revenues!

In [ ]:
# Split the movies by the median rating and median box office revenue
median_rating = full_movie_data_preprocessed['averageRating'].median()
median_box_office = full_movie_data_preprocessed['box_office_revenue'].median()

# Add a column to the DataFrame to indicate if the movie has a high rating/box office revenue
full_movie_data_preprocessed['high_rating'] = full_movie_data_preprocessed['averageRating'] > median_rating
full_movie_data_preprocessed['high_box_office'] = full_movie_data_preprocessed['box_office_revenue'] > median_box_office

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

histogram_movies(full_movie_data_preprocessed, hue="high_rating", axes=axes[0])
histogram_movies(full_movie_data_preprocessed, hue="high_box_office", axes=axes[1])

fig.suptitle("Comparison of movies with high/low ratings (first row) and high/low box office revenue (second row)", fontsize=16)
fig.tight_layout()


First, we see that the box office was correctly split by low/high revenue (see second plot of the second row).

Futhermore, we can observe a slight difference in the runtime distribution between high and low rated/revenue movies.

Now let's look at the average ratings and number of votes.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

histogram_movie_ratings(full_movie_data_preprocessed, hue="high_rating", axes=axes[0])
histogram_movie_ratings(full_movie_data_preprocessed, hue="high_box_office", axes=axes[1])

fig.suptitle("Comparison of movies with high/low ratings (first row) and high/low box office revenue (second row)", fontsize=16)
fig.tight_layout()

Again, we see that the data was correctly split into low and high rating (see first plot of the first row).

Further, we can observe that high rated/revenue movies have more votes.

Now, let's look at the categorical features.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

count_movies(full_movie_data_preprocessed, hue="high_rating", axes=axes[0])
count_movies(full_movie_data_preprocessed, hue="high_box_office", axes=axes[1])

fig.suptitle("Comparison of movies with high/low ratings (first row) and high/low box office revenue (second row)", fontsize=16)
fig.tight_layout()

We can see that there are more higher rated dramas than lower rated one, but there are less high box office revenue dramas as high revenue one.

Futhermore, we see there are more lower rated comedies but they usually have higher box office revenues.

Now let's have a look at the difference in lead actors' attributes between high and low rating movies.

In [ ]:
# Merge the movie data with the lead actors data
merged = pd.merge(full_movie_data_preprocessed, lead_actors_data_on_subset_movie, left_on='freebase_movie_id', right_on='freebase_movie_id')

# Split the movies by the median rating and median box office revenue
median_rating = merged['averageRating'].median()
median_box_office = merged['box_office_revenue'].median()

# Add a column to the DataFrame to indicate if the movie has a high rating/box office revenue
merged['high_rating'] = merged['averageRating'] > median_rating
merged['high_box_office'] = merged['box_office_revenue'] > median_box_office

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

histogram_actors(merged, hue='high_rating', axes=axes[0])
histogram_actors(merged, hue='high_box_office', axes=axes[1])

fig.suptitle("Comparison of movies with high/low ratings (first row) and high/low box office revenue (second row)", fontsize=16)
fig.tight_layout()

Not much of a difference here, let's look at the categorical attributes.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

count_actors(merged, "high_rating", axes[0])
count_actors(merged, "high_box_office", axes[1])

fig.suptitle("Comparison of movies with high/low ratings (first row) and high/low box office revenue (second row)", fontsize=16)
fig.tight_layout()

We can see that there are more men in higher rated/revenue movies as in lower rated/revenue movies.
Whereas, it's the opposite for women.

Futhermore, we can see that there are more African Americans in lower rated movies as in high rated movies.
Whereas, it't the opposite for English People.

Interestingly, we see that there are more Affrican Americans in movies with high revenue as in lower box office revenue movies.